<a href="https://colab.research.google.com/github/frknltrk/atasozu-onerici/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy gensim sklearn tensorflow tqdm joblib --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
     |████████████████████████████████| 511.7 MB 6.3 kB/s 
     |████████████████████████████████| 438 kB 58.8 MB/s 
     |████████████████████████████████| 5.8 MB 43.1 MB/s 
     |████████████████████████████████| 1.6 MB 46.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Unins

In [2]:
# https://medium.com/@vkmauryavk/get-execution-time-on-every-cell-execution-on-google-colab-bbcb31e5a262
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 199 µs (started: 2022-06-23 12:30:06 +00:00)


In [3]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Atasozleri")
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/Atasozleri
time: 4.18 ms (started: 2022-06-23 12:30:06 +00:00)


## Hazırlık

In [4]:
import pandas as pd
import re

time: 534 ms (started: 2022-06-23 12:30:06 +00:00)


In [5]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

time: 1.63 ms (started: 2022-06-23 12:30:06 +00:00)


In [18]:
df = pd.read_csv('atasozleri_islenecek.csv', index_col=[0], na_filter=False)

time: 73.7 ms (started: 2022-06-23 12:33:04 +00:00)


## Önişleme

In [10]:
# https://zeyrek.readthedocs.io/en/latest/
!pip install zeyrek nltk
import zeyrek, nltk
nltk.download('punkt')
nltk.download('stopwords')
analyzer = zeyrek.MorphAnalyzer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 930 kB 10.1 MB/s 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


time: 13.5 s (started: 2022-06-23 12:30:07 +00:00)


In [45]:
def temizle(metin):
  metin = re.sub('İ', 'i', metin) # lower fonksiyonu büyük İ harfinde nedense çalışmıyor
  metin = metin.lower()
  metin = re.sub('[^\w\s]', ' ', metin) # yalnızca kelimeler ve boşluklar (whitespace character) kalacak
  return metin
  
def parcala(metin): # tokenize
  '''Kelimenin kendisi ile son lemma'sını alır.'''
  metin = temizle(metin)
  ogeler = analyzer.lemmatize(metin) # ogeler: tuples
  parcalar = [] # parca: token
  for kelime, adaylar in ogeler:
    if len(kelime)<2 or len(kelime)>20 or kelime in nltk.corpus.stopwords.words('turkish'): continue
    lemma = adaylar[0]
#   lemma = max(adaylar, key=lambda a: len(a[:-3]) if a.endswith("mek") or a.endswith("mak") else len(a))
    if len(lemma)<2 or len(lemma)>20: continue
    lemma = lemma.lower()
    if kelime == lemma:
      parcalar.append(lemma)
    else:
      parcalar.extend([kelime, lemma])
  return parcalar # return ', '.join(parcalar)

time: 52.4 ms (started: 2022-06-23 12:54:09 +00:00)


In [47]:
df['parcalar'] = df['anlam'].apply(parcala)

time: 2min 58s (started: 2022-06-23 12:56:31 +00:00)


## Eğit

### TF-IDF

In [ ]:
# https://radimrehurek.com/gensim/models/tfidfmodel.html
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
kume = df['parcalar'].to_list()
dct = Dictionary(kume)  # fit dictionary
derlem = [dct.doc2bow(parca_listesi) for parca_listesi in kume]  # convert corpus to BoW format
model = TfidfModel(derlem) # fit model

time: 568 ms (started: 2022-06-09 14:40:57 +00:00)


In [ ]:
# https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html
from gensim import similarities
ms_tfidf = similarities.MatrixSimilarity(model[derlem])

def oner_tfidf(cumle, ilk_n=5):  
  """
  Return the database sentences in order of highest cosine similarity relatively to each 
  token of the target sentence. 
  """
  # Embed the query sentence
  cumle_parcalar = parcala(cumle)
  cumle_vektor = dct.doc2bow(cumle_parcalar)
  # Create list with similarity between query and dataset
  benzerler = ms_tfidf[cumle_vektor]
  # Best cosine distance for each token independantly
  sonuclar = sorted(enumerate(benzerler), key=lambda item: -item[1])
  # Return the indices of top k result
  return [indis for indis, skor in sonuclar][:ilk_n]

time: 734 ms (started: 2022-06-09 14:40:58 +00:00)


In [ ]:
ORNEK_SORGU = "zamanında dedenin yaptığı hatalar bugün senin"

time: 781 µs (started: 2022-06-09 14:40:58 +00:00)


In [ ]:
eniyi_indisler = oner_tfidf(ORNEK_SORGU)
df.iloc[eniyi_indisler]

,title,anlam,parcalar
pageid,,,
231312,Herkesin geçtiği köprüden sen de geç,# Sana uygun görünmese bile herkesin yaptığı i...,"[sana, uygun, görünmese, görünmek, bile, herke..."
231299,"Hayır dile eşine, hayır gele başına",# Sen başkaları için iyi şeyler dile ve yap ki...,"[sen, başkaları, başka, iyi, şeyler, şey, dile..."
181572,Gelen gideni aratır,"# ""Tanışılan kişiler,unutulanlardan daha büyük...","[tanışılan, tanışmak, kişiler, kişi, unutulanl..."
181516,"Aba vakti aba, yaba vakti yaba alan yanılmaz","# ""Her şey zamanında yapılırsa kişi kazançlı o...","[zamanında, yapılırsa, yapmak, kişi, kazançlı,..."
230355,"Ada bana, adayım sana",# Sen bir kimse için fedakarlıkta bulunursan o...,"[sen, bir, kimse, fedakarlıkta, fedakâr, bulun..."


time: 63.9 ms (started: 2022-06-09 14:40:58 +00:00)


### Word2Vec

In [ ]:
from gensim.models.word2vec import Word2Vec

# Create model
w2v_model = Word2Vec(vector_size=300, min_count=0, workers=8)
# Prepare vocab
w2v_model.build_vocab(df['parcalar'].values)
# Train
w2v_model.train(df['parcalar'].values, total_examples=w2v_model.corpus_count, epochs=30)

(1378610, 1509240)

time: 7.08 s (started: 2022-06-09 14:41:25 +00:00)


In [ ]:
import numpy as np
def is_word_in_model(word, model):
  """
  Check on individual words ``word`` that it exists in ``model``.
  """
  assert type(model).__name__ == 'KeyedVectors'
  is_in_vocab = word in model.key_to_index.keys()
  return is_in_vocab

def predict_w2v(query_sentence, dataset, model, topk=5):
  query_sentence = parcala(query_sentence)
  in_vocab_list, best_index = [], [0]*topk
  for w in query_sentence:
    # remove unseen words from query sentence
    if is_word_in_model(w, model.wv):
      in_vocab_list.append(w)
  # Retrieve the similarity between two words as a distance
  if len(in_vocab_list) > 0:
    sim_mat = np.zeros(len(dataset))  # TO DO
    for i, data_sentence in enumerate(dataset):
      if data_sentence:
        sim_sentence = model.wv.n_similarity(in_vocab_list, data_sentence)
      else:
        sim_sentence = 0
      sim_mat[i] = np.array(sim_sentence)
    # Take the five highest norm
    best_index = np.argsort(sim_mat)[::-1][:topk]
  return best_index

time: 43.8 ms (started: 2022-06-09 14:41:58 +00:00)


In [ ]:
ORNEK_SORGU = "zamanında dedenin yaptığı hatalar bugün senin"

time: 797 µs (started: 2022-06-09 14:42:01 +00:00)


In [ ]:
best_index = predict_w2v(ORNEK_SORGU, df['parcalar'].values, w2v_model)    
df.iloc[best_index]

,title,anlam,parcalar
pageid,,,
229898,Sükut ikrardan gelir,# Bir kişiye: Sen şöyle bir iş yaptın mı? (yap...,"[bir, kişiye, kişi, sen, şöyle, bir, iş, yaptı..."
230880,"Dağ başına harman yapma, savurursun yel için; ...","# Yapacağın iyi bir işi, kimi etkenlerle ziyan...","[yapacağın, iyi, bir, işi, iş, kimi, etkenlerl..."
181631,Papaz her gün pilav yemez,# Bir işi her gün aynı kişiye yaptırırsan o ön...,"[bir, işi, iş, gün, aynı, kişiye, kişi, yaptır..."
229875,"Şaşkın ördek başını bırakır, kıçından dalar","# Ne yaptığını bilmeyen kişi, işi tersinden yü...","[yaptığını, yapmak, bilmeyen, bilmek, kişi, iş..."
370054,"Hanım kırarsa kaza, halayık kırarsa ceza.",# Buyurucu durumunda olanların yaptığı yanlışl...,"[buyurucu, buyuru, durumunda, durum, olanların..."


time: 683 ms (started: 2022-06-09 14:42:02 +00:00)


### Transformer: BERT

In [20]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 4.4 MB 41.5 MB/s 
     |████████████████████████████████| 1.2 MB 50.9 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 6.6 MB 61.1 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=dda7a9f2f5cfcef2169f9873b31600cb47eb2657ed21a1545cf7ff9b25fc01b5
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
time: 12.8 s (started: 2022-06-23 12:35:30 +00:00)


In [21]:
# https://www.sbert.net/docs/quickstart.html
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('dbmdz/bert-base-turkish-uncased')

/usr/local/lib/python3.7/dist-packages/huggingface_hub/snapshot_download.py:11: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/dbmdz_bert-base-turkish-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


time: 35.7 s (started: 2022-06-23 12:35:43 +00:00)


In [49]:
parcalar_string = df['parcalar'].apply(lambda parcalar: ' '.join(parcalar)).values
corpus_embs = model.encode(parcalar_string)

time: 5min 19s (started: 2022-06-23 13:00:10 +00:00)


In [50]:
# https://www.sbert.net/examples/applications/semantic-search/README.html
from sentence_transformers import util

def eniyileri_getir(sorgu, ilk_n=5):
  query_emb = model.encode(sorgu)
  cos_benz = util.cos_sim(query_emb, corpus_embs)[0]
  return cos_benz.topk(ilk_n).indices

time: 3.67 ms (started: 2022-06-23 13:05:30 +00:00)


In [52]:
ORNEK_SORGU = "gün gece arkanda bırakmak unutmak sıkıntı"
df.iloc[eniyileri_getir(ORNEK_SORGU)]

,title,wikitext,anlam,muhtemel aynılar,parcalar
pageid,,,,,
229851,Çivi çıkar ama yeri kalır,== Türkçe ==\n\n===Atasözü===\n{{tr-atasözü}}\...,# Başkasına yaptığın bir fenalığı kaldırıp gid...,,"[başkasına, başka, yaptığın, yapmak, bir, fena..."
230599,"Ayda bir gel dostuna, kalksın ayak üstüne; gün...",==Türkçe==\n===Atasözü===\n{{başlık başı|tr|at...,"# Ne denli yakın dostun olursa olsun, ona her ...",,"[denli, yakın, dostun, dost, olursa, olur, ols..."
231476,Kara gün kararıp kalmaz,==Türkçe==\n===Atasözü===\n{{başlık başı|tr|at...,"# İnsanın sıkıntılı zamanı sürüp gitmez, arkas...",,"[insanın, insan, sıkıntılı, sıkıntı, zamanı, z..."
230881,"Dağ başına kış gelir, insanın başına iş gelir",==Türkçe==\n===Atasözü===\n{{başlık başı|tr|at...,# Dağ başında kışın nasıl fırtına eksik olmazs...,,"[dağ, başında, baş, kışın, kış, fırtına, eksik..."
229910,"Sık gidersen dostuna, yatar arka üstüne",==Türkçe==\n===Atasözü===\n{{başlık başı|tr|at...,"# Ne denli candan olursa olsun, dostuna çok sı...",,"[denli, candan, olursa, olur, olsun, olmak, do..."


time: 242 ms (started: 2022-06-23 13:08:51 +00:00)


## Sakla
*Sakla samanı gelir zamanı.*

In [ ]:
import torch
torch.save(corpus_embs, 'proverbs_embeddings.pt')

['modeller/bert_model_pipeline.pkl']

time: 3 s (started: 2022-06-08 16:48:33 +00:00)


## Kaynaklar
- https://github.com/bledem/movie_recommendation
- StackOverFlow
- https://github.com/vngrs-ai/vnlp